In [1]:
using Revise

In [21]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using Gurobi
using Flux
using ProgressMeter
using MLDataPattern
using JLD2
using JuMP
using IterTools: subsets
using Plots; gr()

Plots.GRBackend()

In [3]:
import LCPSim
import LearningMPC
import Hoppers
import FluxExtensions

In [4]:
robot = Hoppers.Hopper()
mvis = MechanismVisualizer(robot.mechanism, URDFVisuals(Hoppers.urdf))
IJuliaCell(mvis)

Listening on 127.0.0.1:7012...
zmq_url=tcp://127.0.0.1:6012
web_url=http://127.0.0.1:7012/static/


MeshCat.IJuliaCell(MeshCat.ViewerWindow(ZMQ.Context(Ptr{Void} @0x000000000617b6a0, ZMQ.Socket[ZMQ.Socket(Ptr{Void} @0x00000000065c0530, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000006793d70, 79, (1, 0), Condition(Any[]), (false, false), 0))]), ZMQ.Socket(Ptr{Void} @0x00000000065c0530, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000006793d70, 79, (1, 0), Condition(Any[]), (false, false), 0)), "http://127.0.0.1:7012/static/", "tcp://127.0.0.1:6012", MeshCat.ZMQServer.ZMQWebSocketBridge(ip"127.0.0.1", Set{WebSockets.WebSocket}(), MeshCat.SceneTrees.SceneNode(Nullable{Array{UInt8,1}}(), Nullable{Array{UInt8,1}}(), Dict("meshcat"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0xa8, 0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74], Dict("world"=>MeshCat.SceneTrees.SceneNode(#NULL, #NULL, Dict("floor"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x66, 0x6c, 0x6f, 0x6f, 0x72], Dict("floor"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6c, 0x6f, 0x6f, 0x72, 0x2f, 0x66, 0x6c, 0x6f, 0x6f, 0x72], Dict("geometry_1"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String,MeshCat.SceneTrees.SceneNode}()))),"core"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x6c, 0x6f, 0x6f, 0x72, 0x2f, 0x63, 0x6f, 0x72, 0x65], Dict("after_base_z"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x65, 0x72, 0x5f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x7a], Dict("geometry_2"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String,MeshCat.SceneTrees.SceneNode}()))),"foot"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x63, 0x6f, 0x72, 0x65, 0x2f, 0x66, 0x6f, 0x6f, 0x74], Dict("after_foot_extension"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x65, 0x78, 0x74, 0x65, 0x6e, 0x73, 0x69, 0x6f, 0x6e], Dict("geometry_3"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String,MeshCat.SceneTrees.SceneNode}()))))))))))))))), 7012, HttpServer.Server(HttpServer.HttpHandler(MeshCat.ZMQServer.handle_file_request, Base.TCPServer(RawFD(53) active), Dict{String,Function}(Pair{String,Function}("error", HttpServer.#1),Pair{String,Function}("listen", HttpServer.#3))), WebSockets.WebSocketHandler(MeshCat.ZMQServer.#5)), ZMQ.Context(Ptr{Void} @0x00000000064f3740, ZMQ.Socket[ZMQ.Socket(Ptr{Void} @0x000000000652dff0, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000006340b70, 60, (1, 0), Condition(Any[]), (false, false), 0)), ZMQ.Socket(Ptr{Void} @0x000000000555fbd0, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000006299640, 61, (1, 0), Condition(Any[]), (false, false), 0)), ZMQ.Socket(Ptr{Void} @0x000000000633b070, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000006821030, 62, (1, 0), Condition(Any[]), (false, false), 0)), ZMQ.Socket(Ptr{Void} @0x0000000006531860, Base.Filesystem._FDWatcher(Ptr{Void} @0x00000000061c64a0, 63, (1, 0), Condition(Any[]), (false, false), 0)), ZMQ.Socket(Ptr{Void} @0x

In [5]:
function contact_state(x::MechanismState, env::LCPSim.Environment, threshold=1e-2)
    map(env.contacts) do c
        body, pt, obstacle = c
        ϕ = LCPSim.separation(obstacle, transform_to_root(x, pt.frame) * pt)
        ϕ < threshold
    end
end     

contact_state (generic function with 2 methods)

In [6]:
function create_net()
    net = Chain(
        Dense(4, 32, elu),
        Dense(32, 32, elu),
        Dense(32, 1)
    )
    loss = (x, lb, ub) -> begin
        y = net(x)
        sum(ifelse.(y .< lb, lb .- y, ifelse.(y .> ub, y .- ub, 0 .* y)))
    end
    net, loss
end

create_net (generic function with 1 method)

In [7]:
net, loss = create_net()
net_params = params(net)
optimizer = Flux.Optimise.ADAM(net_params)

mpc_params = LearningMPC.MPCParams(
    Δt=0.05,
    horizon=10,
    mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, 
        TimeLimit=5, 
        MIPGap=1e-1,
        FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

xstar = Hoppers.nominal_state(robot)
contact_index_sets = subsets(1:length(robot.environment.contacts))
B_matrices = Dict(map(contact_index_sets) do idxs
    contacts = robot.environment.contacts[idxs]
    contact_state = fill(false, length(robot.environment.contacts))
    contact_state[idxs] = true
    pts = unique(Point3D[p[2] for p in contacts])
    ustar = LearningMPC.nominal_input(xstar, pts)
    Jc = LCPSim.ContactLQR.contact_jacobian(xstar, pts)
    A, B, c = LCPSim.ContactLQR.contact_linearize(xstar, ustar, Jc)
    contact_state => B
end)

Q, R = Hoppers.default_costs(robot)
foot = findbody(robot.mechanism, "foot")
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, mpc_params.Δt, [Point3D(default_frame(foot), 0., 0., 0.)])
lqrsol.S .= 1 ./ mpc_params.Δt .* Q

x_net = MechanismState(robot.mechanism)
tangent_net = FluxExtensions.TangentPropagator(net)
net_controller = state -> begin
    set_configuration!(x_net, configuration(state))
    set_velocity!(x_net, velocity(state))
    value, jac = tangent_net(Vector(state))
    cs = contact_state(x_net, robot.environment)
    B = B_matrices[cs]
    u = vec(-inv(R) * B' * Flux.Tracker.data(jac)')
end

mpc_controller = LearningMPC.MPCController(robot.mechanism, 
    robot.environment, mpc_params, lqrsol, 
    [net_controller, lqrsol]);

sample_sink = LearningMPC.MPCSampleSink{Float64}(true)
playback_sink = LearningMPC.PlaybackSink(mvis, mpc_params.Δt)

mpc_controller.callback = LearningMPC.call_each(
    sample_sink,
#     playback_sink,
)

live_viewer = LearningMPC.live_viewer(mvis)


dagger_controller = LearningMPC.call_each(
    LearningMPC.dagger_controller(
        mpc_controller,
        net_controller,
        0.2),
    live_viewer
    )

dataset = LearningMPC.Dataset(lqrsol)

x_init = MechanismState(robot.mechanism)
set_configuration!(x_init, [1, 1])

function collect_into!(data::Vector{<:LearningMPC.Sample})
    empty!(sample_sink)
    LearningMPC.randomize!(x0, x_init, 0.0, 1.5)
    results = LCPSim.simulate(x0, 
        dagger_controller,
        robot.environment, mpc_params.Δt, 50, 
        mpc_params.lcp_solver);
    append!(data, sample_sink.samples)
end

x0 = MechanismState{Float64}(robot.mechanism)

features(s::LearningMPC.Sample) = (s.state, s.mip.objective_bound, s.mip.objective_value)

features (generic function with 1 method)

In [8]:
datasets = Vector{LearningMPC.Dataset{Float64}}()
all_training_data = Vector{Tuple{Vector{Float64}, Float64, Float64}}()
all_validation_data = Vector{Tuple{Vector{Float64}, Float64, Float64}}()
losses = Vector{Tuple{Float64, Float64}}()

@showprogress for i in 1:20
    dataset = LearningMPC.Dataset(lqrsol)
    for i in 1:2
        collect_into!(dataset.training_data)
    end
    collect_into!(dataset.testing_data)
    collect_into!(dataset.validation_data)
    append!(all_training_data, features.(dataset.training_data))
    append!(all_validation_data, features.(dataset.validation_data))
    
    @time for i in 1:100
        Flux.train!(loss, shuffleobs(all_training_data), optimizer)
        push!(losses, 
            (mean(xy -> Flux.Tracker.data(loss(xy...)), 
                  all_training_data),
             mean(xy -> Flux.Tracker.data(loss(xy...)), 
                  all_validation_data)))
    end
    push!(datasets, dataset)
    
    jldopen("hopper.jld2", "w") do file
        file["datasets"] = datasets
        file["net"] = net
        file["lqrsol"] = lqrsol
        file["mpc_params"] = Dict(
            "Δt" => mpc_params.Δt,
            "horizon" => mpc_params.horizon,
        )
        file["all_training_data"] = all_training_data
        file["all_validation_data"] = all_validation_data
        file["losses"] = losses
    end
    
    plt = plot(first.(losses), label="training")
    plot!(plt, last.(losses), label="validation")
    ylims!(plt, (0, ylims(plt)[2]))
    display(plt)
end

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 150 
 
<polyline clip-path="url(#clip0402)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.8198,11.811 41.3197,13.1424 46.8197,14.504 52.3196,15.7247 57.8196,17.0213 63.3195,18.2781 68.8195,19.7282 74.3194,21.246 79.8194,22.8498 85.3193,24.6978 
 90.8193,26.2445 96.3193,27.359 101.819,28.7945 107.319,30.184 112.819,31.7314 118.319,33.5369 123.819,35.2707 129.319,37.1086 134.819,39.0801 140.319,41.4772 
 145.819,43.8013 151.319,46.4425 156.819,49.1344 162.319,52.1447 167.819,55.7901 173.319,58.745 178.819,60.8162 184.318,62.5482 189.818,64.4058 195.318,66.7208 
 200.818,68.966 206.318,70.6485 211.818,72.5964 217.318,75.1406 222.818,77.3206 228.318,79.6223 233.818,82.6747 239.318,85.0855 244.818,88.0652 250.318,91.0253 
 255.818,94.0177 261.318,97.5637 266.818,100.166 272.318,102.487 277.818,105.027 283.318,107.717 288.818,110.299 294.317,112.61 299.817,114.818 305.317,117.959 
 310.817,120.666 316.317,123.6 321.817,127.331 327.317,130.53 332.817,134.693 338.317,138.438 343.817,141.798 349.317,146.554 354.817,149.705 360.317,153.945 
 365.817,158.99 371.317,162.662 376.817,167.205 382.317,172.533 387.817,178.06 393.317,185.042 398.817,191.001 404.317,198.101 409.816,204.182 415.316,210.6 
 420.816,217.679 426.316,224.263 431.816,231.093 437.316,239.68 442.816,242.242 448.316,243.258 453.816,244.998 459.316,246.695 464.816,249.294 470.316,250.912 
 475.816,252.022 481.316,252.919 486.816,255.252 492.316,255.696 497.816,257.991 503.316,259.118 508.816,260.685 514.316,261.745 519.815,263.78 525.315,264.944 
 530.815,266.659 536.315,268.455 541.815,270.149 547.315,271.659 552.815,273.033 558.315,274.313 563.815,274.748 569.315,276.97 574.815,277.671 580.315,278.374 
 
 "/>
<polyline clip-path="url(#clip0402)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.8198,43.3572 41.3197,44.8667 46.8197,46.4329 52.3196,47.8535 57.8196,49.3076 63.3195,50.6862 68.8195,52.2841 74.3194,53.956 79.8194,55.6669 85.3193,57.6943 
 90.8193,59.683 96.3193,61.2626 101.819,63.3959 107.319,65.3588 112.819,67.5742 118.319,70.1918 123.819,72.6432 129.319,74.4182 134.819,75.8042 140.319,77.169 
 145.819,78.5252 151.319,80.1145 156.819,81.6893 162.319,83.3323 167.819,85.3768 173.319,87.2629 178.819,89.181 184.318,90.8085 189.818,92.6281 195.318,94.8157 
 200.818,96.9215 206.318,98.5508 211.818,100.435 217.318,102.946 222.818,105.101 228.318,107.346 233.818,110.349 239.318,112.628 244.818,115.576 250.318,118.487 
 255.818,121.436 261.318,124.93 266.818,127.873 272.318,130.397 277.818,133.012 283.318,135.868 288.818,138.653 294.317,140.942 299.817,142.956 305.317,146.342 
 310.817,149.044 316.317,152.14 321.817,156.096 327.317,159.366 332.817,164.139 338.317,168.133 343.817,171.905 349.317,177.038 354.817,180.193 360.317,184.625 
 365.817,189.812 371.317,193.576 376.817,198.192 382.317,203.597 387.817,209.417 393.317,216.723 398.817,221.125 404.317,217.738 409.816,213.51 415.316,209.873 
 420.816,205.258 426.316,201.704 431.816,197.406 437.316,191.691 442.816,186.274 448.316,185.678 453.816,187.25 459.316,188.44 464.816,192.678 470.316,194.208 
 475.816,193.667 481.316,192.571 486.816,195.396 492.316,198.45 497.816,197.229 503.316,196.795 508.816,197.968 514.316,201.404 519.815,200.411 525.315,200.242 
 530.815,201.771 536.315,203.601 541.815,205.639 547.315,205.835 552.815,206.131 558.315,207.222 563.815,210.21 569.315,209.789 574.815,209.785 580.315,213.128 
 
 "/>
 
 
 
 
 training 
 
 
 
 validation

  2.314121 seconds (1.71 M allocations: 94.053 MiB, 2.30% gc time)


Progress:   5%|██                                       |  ETA: 0:14:15

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 150 
 
<polyline clip-path="url(#clip0602)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.8198,11.811 38.5559,13.1424 41.2921,14.504 44.0283,15.7247 46.7644,17.0213 49.5006,18.2781 52.2367,19.7282 54.9729,21.246 57.709,22.8498 60.4452,24.6978 
 63.1814,26.2445 65.9175,27.359 68.6537,28.7945 71.3898,30.184 74.126,31.7314 76.8621,33.5369 79.5983,35.2707 82.3344,37.1086 85.0706,39.0801 87.8068,41.4772 
 90.5429,43.8013 93.2791,46.4425 96.0152,49.1344 98.7514,52.1447 101.488,55.7901 104.224,58.745 106.96,60.8162 109.696,62.5482 112.432,64.4058 115.168,66.7208 
 117.904,68.966 120.641,70.6485 123.377,72.5964 126.113,75.1406 128.849,77.3206 131.585,79.6223 134.321,82.6747 137.058,85.0855 139.794,88.0652 142.53,91.0253 
 145.266,94.0177 148.002,97.5637 150.738,100.166 153.475,102.487 156.211,105.027 158.947,107.717 161.683,110.299 164.419,112.61 167.155,114.818 169.891,117.959 
 172.628,120.666 175.364,123.6 178.1,127.331 180.836,130.53 183.572,134.693 186.308,138.438 189.045,141.798 191.781,146.554 194.517,149.705 197.253,153.945 
 199.989,158.99 202.725,162.662 205.461,167.205 208.198,172.533 210.934,178.06 213.67,185.042 216.406,191.001 219.142,198.101 221.878,204.182 224.615,210.6 
 227.351,217.679 230.087,224.263 232.823,231.093 235.559,239.68 238.295,242.242 241.032,243.258 243.768,244.998 246.504,246.695 249.24,249.294 251.976,250.912 
 254.712,252.022 257.448,252.919 260.185,255.252 262.921,255.696 265.657,257.991 268.393,259.118 271.129,260.685 273.865,261.745 276.602,263.78 279.338,264.944 
 282.074,266.659 284.81,268.455 287.546,270.149 290.282,271.659 293.019,273.033 295.755,274.313 298.491,274.748 301.227,276.97 303.963,277.671 306.699,278.374 
 309.435,253.125 312.172,262.482 314.908,272.503 317.644,279.104 320.38,284.833 323.116,289.35 325.852,292.852 328.589,295.583 331.325,297.597 334.061,299.758 
 336.797,300.942 339.533,301.752 342.269,302.027 345.005,302.229 347.742,302.735 350.478,303.446 353.214,302.386 355.95,302.812 358.686,303.259 361.422,303.793 
 364.159,302.795 366.895,302.576 369.631,304.565 372.367,304.49 375.103,305.239 377.839,303.232 380.576,305.226 383.312,304.919 386.048,305.264 388.784,306.293 
 391.52,306.528 394.256,306.504 396.992,306.634 399.729,307.265 402.465,306.633 405.201,306.983 407.937,306.857 410.673,307.638 413.409,307.749 416.146,308.798 
 418.882,308.968 421.618,308.957 424.354,309.823 427.09,310.691 429.826,310.752 432.563,311.699 435.299,311.893 438.035,311.91 440.771,313.03 443.507,313.949 
 446.243,313.69 448.979,314.354 451.716,314.885 454.452,314.839 457.188,315.526 459.924,316.338 462.66,316.823 465.396,317.103 468.133,317.225 470.869,316.893 
 473.605,317.558 476.341,318.716 479.077,319.682 481.813,317.923 484.549,320.827 487.286,321.057 490.022,320.724 492.758,320.672 495.494,322.801 498.23,323.138 
 500.966,323.538 503.703,324.108 506.439,323.866 509.175,323.792 511.911,324.453 514.647,324.011 517.383,325 520.12,325.762 522.856,324.394 525.592,326.3 
 528.328,326.5 531.064,326.904 533.8,326.746 536.536,327.438 539.273,326.73 542.009,327.908 544.745,327.738 547.481,328.462 550.217,328.224 552.953,327.848 
 555.69,328.065 558.426,328.627 561.162,329.448 563.898,329.099 566.634,330.322 569.37,330.359 572.106,330.761 574.843,331.003 577.579,331.381 580.315,331.706 
 
 "/>
<polyline clip-path="url(#clip0602)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.8198,43.3572 38.5559,44.8667 41.2921,46.4329 44.0283,47.8535 46.7644,49.3076 49.5006,50.6862 52.2367,52.2841 54.9729,53.956 57.709,55.6669 60.4452,57.6943 
 63.1814,59.683 65.9175,61.2626 68.6537,63.3959 71.3898,65.3588 74.126,67.5742 76.8621,70.1918 79.5983,72.6432 82.3344,74.4182 85.0706,75.8042 87.8068,77.169 
 90.5429,78.5252 93.2791,80.1145 96.0152,81.6893 98.7514,83.3323 101.48

  1.313859 seconds (1.66 M allocations: 108.187 MiB, 2.96% gc time)


...

 17.415178 seconds (22.18 M allocations: 1.411 GiB, 3.04% gc time)


Progress:  95%|███████████████████████████████████████  |  ETA: 0:00:16

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 150 
 
<polyline clip-path="url(#clip4202)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.8198,11.811 36.0922,13.1424 36.3646,14.504 36.6369,15.7247 36.9093,17.0213 37.1817,18.2781 37.4541,19.7282 37.7265,21.246 37.9989,22.8498 38.2712,24.6978 
 38.5436,26.2445 38.816,27.359 39.0884,28.7945 39.3608,30.184 39.6332,31.7314 39.9055,33.5369 40.1779,35.2707 40.4503,37.1086 40.7227,39.0801 40.9951,41.4772 
 41.2675,43.8013 41.5398,46.4425 41.8122,49.1344 42.0846,52.1447 42.357,55.7901 42.6294,58.745 42.9018,60.8162 43.1741,62.5482 43.4465,64.4058 43.7189,66.7208 
 43.9913,68.966 44.2637,70.6485 44.5361,72.5964 44.8085,75.1406 45.0808,77.3206 45.3532,79.6223 45.6256,82.6747 45.898,85.0855 46.1704,88.0652 46.4428,91.0253 
 46.7151,94.0177 46.9875,97.5637 47.2599,100.166 47.5323,102.487 47.8047,105.027 48.0771,107.717 48.3494,110.299 48.6218,112.61 48.8942,114.818 49.1666,117.959 
 49.439,120.666 49.7114,123.6 49.9837,127.331 50.2561,130.53 50.5285,134.693 50.8009,138.438 51.0733,141.798 51.3457,146.554 51.618,149.705 51.8904,153.945 
 52.1628,158.99 52.4352,162.662 52.7076,167.205 52.98,172.533 53.2523,178.06 53.5247,185.042 53.7971,191.001 54.0695,198.101 54.3419,204.182 54.6143,210.6 
 54.8867,217.679 55.159,224.263 55.4314,231.093 55.7038,239.68 55.9762,242.242 56.2486,243.258 56.521,244.998 56.7933,246.695 57.0657,249.294 57.3381,250.912 
 57.6105,252.022 57.8829,252.919 58.1553,255.252 58.4276,255.696 58.7,257.991 58.9724,259.118 59.2448,260.685 59.5172,261.745 59.7896,263.78 60.0619,264.944 
 60.3343,266.659 60.6067,268.455 60.8791,270.149 61.1515,271.659 61.4239,273.033 61.6962,274.313 61.9686,274.748 62.241,276.97 62.5134,277.671 62.7858,278.374 
 63.0582,253.125 63.3305,262.482 63.6029,272.503 63.8753,279.104 64.1477,284.833 64.4201,289.35 64.6925,292.852 64.9649,295.583 65.2372,297.597 65.5096,299.758 
 65.782,300.942 66.0544,301.752 66.3268,302.027 66.5992,302.229 66.8715,302.735 67.1439,303.446 67.4163,302.386 67.6887,302.812 67.9611,303.259 68.2335,303.793 
 68.5058,302.795 68.7782,302.576 69.0506,304.565 69.323,304.49 69.5954,305.239 69.8678,303.232 70.1401,305.226 70.4125,304.919 70.6849,305.264 70.9573,306.293 
 71.2297,306.528 71.5021,306.504 71.7744,306.634 72.0468,307.265 72.3192,306.633 72.5916,306.983 72.864,306.857 73.1364,307.638 73.4087,307.749 73.6811,308.798 
 73.9535,308.968 74.2259,308.957 74.4983,309.823 74.7707,310.691 75.0431,310.752 75.3154,311.699 75.5878,311.893 75.8602,311.91 76.1326,313.03 76.405,313.949 
 76.6774,313.69 76.9497,314.354 77.2221,314.885 77.4945,314.839 77.7669,315.526 78.0393,316.338 78.3117,316.823 78.584,317.103 78.8564,317.225 79.1288,316.893 
 79.4012,317.558 79.6736,318.716 79.946,319.682 80.2183,317.923 80.4907,320.827 80.7631,321.057 81.0355,320.724 81.3079,320.672 81.5803,322.801 81.8526,323.138 
 82.125,323.538 82.3974,324.108 82.6698,323.866 82.9422,323.792 83.2146,324.453 83.4869,324.011 83.7593,325 84.0317,325.762 84.3041,324.394 84.5765,326.3 
 84.8489,326.5 85.1213,326.904 85.3936,326.746 85.666,327.438 85.9384,326.73 86.2108,327.908 86.4832,327.738 86.7556,328.462 87.0279,328.224 87.3003,327.848 
 87.5727,328.065 87.8451,328.627 88.1175,329.448 88.3899,329.099 88.6622,330.322 88.9346,330.359 89.207,330.761 89.4794,331.003 89.7518,331.381 90.0242,331.706 
 90.2965,273.749 90.5689,283.741 90.8413,288.354 91.1137,298.705 91.3861,298.044 91.6585,295.578 91.9308,298.861 92.2032,300.479 92.4756,300.443 92.748,298.48 
 93.0204,301.092 93.2928,302.357 93.5651,301.904 93.8375,300.652 94.1099,302.062 94.3823,299.65 94.6547,302.695 94.9271,302.211 95.1995,302.824 95.4718,301.337 
 95.7442,300.766 96.0166,303.697 96.289,300.524 96.5614,302.965 96.8338,304.202 97.1061,304.655 97.3785,302.682 97.6509,303.279 97.9233,303.621 98.1957,303.346 
 98.4681,305.267 98.7404,30

 18.269266 seconds (23.23 M allocations: 1.478 GiB, 3.00% gc time)


Progress: 100%|█████████████████████████████████████████| Time: 0:05:23


In [9]:
# optimizer = Flux.ADAM(net_params)
# @time for i in 1:1000
#     Flux.train!(loss, shuffleobs(all_training_data), optimizer)
#     push!(losses, 
#         (mean(xy -> Flux.Tracker.data(loss(xy...)), 
#               all_training_data),
#          mean(xy -> Flux.Tracker.data(loss(xy...)), 
#               all_validation_data)))
# end

# plt = plot(first.(losses), label="training")
# plot!(plt, last.(losses), label="validation")
# ylims!(plt, (0, ylims(plt)[2]))
# plt

In [13]:
net_controller2 = state -> begin
    set_configuration!(x_net, configuration(state))
    set_velocity!(x_net, velocity(state))
    value, jac = tangent_net(Vector(state))
    cs = fill(true, length(robot.environment.contacts))
    B = B_matrices[cs]
    u = vec(-inv(R) * B' * Flux.Tracker.data(jac)')
end

(::#25) (generic function with 1 method)

In [14]:
x0 = MechanismState{Float64}(robot.mechanism)
# LearningMPC.randomize!(x0, xstar, 0.0, 1.0)
set_configuration!(x0, [1, 1])
results = LCPSim.simulate(x0, 
    net_controller2,
    robot.environment, mpc_params.Δt, 100, 
    mpc_params.lcp_solver,
    termination=x -> false);
LearningMPC.playback(mvis, results, 0.05)

In [36]:
net_qp_controller = state -> begin
    set_configuration!(x_net, configuration(state))
    set_velocity!(x_net, velocity(state))
    value, jac = tangent_net(Vector(state))
    m = Model(solver=mpc_params.mip_solver)
    _, results = LCPSim.optimize(x_net, robot.environment, mpc_params.Δt, 1, m)
    @assert length(results) == 1
    x = results[1].state.state
    u = results[1].input
    q = vec(Flux.Tracker.data(jac))
    @objective m Min u' * lqrsol.R * u + x' * lqrsol.Q * x + q' * x
    solve(m)
    getvalue(results[1].input)
end
    

(::#69) (generic function with 1 method)

In [37]:
x0 = MechanismState{Float64}(robot.mechanism)
# LearningMPC.randomize!(x0, xstar, 0.0, 1.0)
set_configuration!(x0, [1, 1])
results = LCPSim.simulate(x0, 
    net_qp_controller,
    robot.environment, mpc_params.Δt, 100, 
    mpc_params.lcp_solver,
    termination=x -> false);
LearningMPC.playback(mvis, results, 0.05)

In [ ]:
k